# 6. Pytorch Lightning

딥러닝 실험을 구현하기 위해서는 신경망(Neural Network)와 같은 모델 코드 외에도 그 시스템을 만들고 실험을 수행하기 위한 많은 엔지니어링 코드가 필요합니다. 이러한 코드들은 직접 짜는게 귀찮을뿐더러 남이 짠 코드를 읽을 때도 코드를 분석하기 어렵게 만듭니다. 그런데 사실 딥러닝에서의 많은 엔지니어링 코드는 모델이 달라져도 그 역할이 비슷비슷한 경우가 많습니다.

머신러닝, 딥러닝 모델 구축을 할 때에 공통된 부분들을 반복해서 작성할 필요 없이 대신 처리해주고, 머신러닝 모델 구축의 탬플릿 코드로써 기능을 하며, 다른 사람이 작성한 코드를 쉽게 볼 수 있도록 공통된 스타일을 갖도록 하고 ,모델의 개별적인 부분은 유연하게 커스터마이징하여 실험할 수 있는 라이브러리가 PyTorch Lightning 입니다.



PyTorch Lightning은 PyTorch에 대한 High-level 인터페이스를 제공하는 오픈소스 Python 라이브러리입니다. PyTorch만으로도 충분히 다양한 AI 모델들을 쉽게 생성할 수 있지만 GPU나 TPU, 그리고 16-bit precision, 분산학습 등 더욱 복잡한 조건에서 실험하게 될 경우, 코드가 복잡해집니다. 따라서 코드의 추상화를 통해, 프레임워크를 넘어 하나의 코드 스타일로 자리 잡기 위해 탄생한 프로젝트가 바로 PyTorch Lightning입니다.

Pytorch Lightning의 장점은 세부적인 High-Level 코드를 작성할때 좀 더 정돈되고 간결화된 코드를 작성할 수 있다는 데에 있습니다.

기존 PyTorch는 DataLoader, Model, optimizer, Training roof 등을 전부 따로따로 코드로 구현을 해야하는데 Pytorch Lightning에서는 Lightning Model class 안에 이 모든 것을 한 번에 구현하도록 되어있습니다. 클래스 내부에 있는 함수명은 똑같이 써야하고 그 목적에 맞게 코딩해야 합니다.

Pytorch Lightning은 크게 Trainer와 Lightning Module로 나누어 살펴볼 수 있습니다.

Lightning Module은 모델 내부의 구조를 설계하는 research & science 클래스라고 생각할 수 있습니다. 모델의 구조나 데이터 전처리, 손실함수 등의 설정을 통해 모델을 초기화 합니다. 실제로 코드에서는 pl.LightningModule 클래스를 상속받아 새로운 LightningModule 클래스를 생성합니다. 기존 PyTorch의 nn.Module과 같은 방식이라고 보시면 됩니다.

Trainer는 모델의 학습을 담당하는 클래스라고 볼 수 있습니다. 모델의 학습 epoch이나 batch 등의 상태뿐만 아니라, 모델을 저장해 로그를 생성하는 부분까지 담당합니다. 실제로 코드에서는 pl.Trainer()라고 정의하면 끝입니다.

결국 두 가지의 큰 클래스를 통해, 복잡한 양의 작업들을 2가지 영역으로 추상화할 수 있게 됩니다. PyTorch Lightning는 다음 명령으로 간단히 설치할 수 있습니다.

pip install pytorch-lightning

또는

conda install pytorch-lightning -c conda-forge

버전에 맞춰서 설치하는 명령어는 다음과 같습니다.

pip install “pytorch-lightning>=1.3"

PyTorch Lightning을 사용하여 딥러닝 모델을 작성하는 순서는

Lightning Module에서 상속된 새로운 Lightning Module 클래스를 작성합니다.
DataLoader를 통해 학습할 데이터를 준비합니다.
Trainer 객체를 만들고, 그 Trainer에 데이터와 Lightning Module 클래스를 주어 학습합니다.
위의 순서대로 간단하게 작성한 코드는 다음과 같습니다.



In [2]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 22.1 MB/s eta 0:00:00


In [3]:
!pip install pytorch-lightning --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pytorch_lightning as pl
print(pl.__version__)


2.0.2


In [1]:
import pytorch_lightning as pl
import os
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

class LitModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28 * 28, 10)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

train_transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(os.getcwd(), train=True, download=True, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_transform = transforms.Compose([transforms.ToTensor()])
val_dataset = datasets.MNIST(os.getcwd(), train=False, download=True, transform=val_transform)
val_loader = DataLoader(val_dataset, batch_size=64)

trainer = pl.Trainer()
model = LitModel()
trainer.fit(model, train_loader, val_loader)


ModuleNotFoundError: ignored